# SOU Exploratory Analyis & File Creation

### Libraries

In [241]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

# Personal Lib for this project
from tools import *

### Testing on One File
- TODO:
    Compile everything into one function

In [242]:
# Path to for rep file
pdf_path = "../data/pdf/March20.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

# Grab the text and figure out use case

In [259]:

def folder_to_dict(folder_path:str, test:bool = False) -> dict:
    '''
    Converts the pdf files to a csv file as long as it follows the same format as the example files.
    By product is printing the name of the players. Will fix later for now its a "Quirk".
    '''
    if test:
        folder = [folder_path]
    else:
        folder = folder_to_list(folder_path)

    i = 0 # used to track the number of files

    for file in folder:
        #print(i, file)
        with pdfplumber.open(file, pages= [1]) as pdf:
            
            # Extract text from each page
            text = ""
            for page in pdf.pages:
                text += page.extract_text();


        matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM T)', text, re.DOTALL)
        matches[0] = matches[0].replace(', ', ',')

        matches_date = re.findall(r'Date: \s*(........)', text)

        stats = matches[0]
        stats = stats.strip().split(sep = '\n')
        split_stats = list(map(str.split, stats))

        opponent = get_opponent(text)
        title = split_stats[0] + ['Opponent'] + ['Date']

        players = split_stats[1:-1]
        #ISSUE HERE

        players = list(map(merge_and_pop, players))
        players = list(map(non_start, players))
        #ISSUE HERE
        if i == 0:
            stat_dict = {key: [] for key in title}

        for player in players:

            for stat, data in zip(title, player):
                stat_dict[stat].append(data)
            # stat_dict['Opponent'].append(opponent)#new
            # stat_dict['Date'].append(matches_date[0])#new
        for name in split_stats[1:-1]:
            stat_dict['Opponent'].append(opponent)
            stat_dict['Date'].append(matches_date[0])
 
        if len(stat_dict['PTS']) != len(stat_dict['Player'] or test == True):
            print(matches_date, 'Length of PTS and Player do not match')
            #return
        i += 1

    return (stat_dict)

In [ ]:
full_data = folder_to_dict('../data/pdf')
pd.

Baird Morgan
McIntyre Bridgette
Schmerbach Emma
McIntyre Meghan
Robbins Clara
Scheppele Sierra
DiGiulio Eliza
Wright Keeley
Scurlock Makayla
Williams Mallory
MCINTYRE BRIDGETTE
SCHMERBACH EMMA
BAIRD MORGAN
MCINTYRE MEGHAN
ROBBINS CLARA
DIGIULIO ELIZA
WILLIAMS MALLORY
WRIGHT KEELEY
SCHEPPELE SIERRA
SCURLOCK MAKAYLA
BAIRD MORGAN
SCHMERBACH EMMA
MCINTYRE BRIDGETTE
MCINTYRE MEGHAN
ROBBINS CLARA
WRIGHT KEELEY
DIGIULIO ELIZA
SCHEPPELE SIERRA
WILLIAMS MALLORY
TEAGUE-PERRY SHAKIA
SCURLOCK MAKAYLA
ZURITA JENIKA
STEVENS ALI
BAIRD MORGAN
SCHMERBACH EMMA
MCINTYRE MEGHAN
MCINTYRE BRIDGETTE
ROBBINS CLARA
WILLIAMS MALLORY
SCURLOCK MAKAYLA
SCHEPPELE SIERRA
TEAGUE-PERRY SHAKIA
WRIGHT KEELEY
DIGIULIO ELIZA
ZURITA JENIKA
STEVENS ALI


# Testing Area

In [ ]:
pd